# Tutorial - Development of a Vespa-based Search Engine Ver. 1.0.2

This tutorial will walk you through major steps for creating a local search engine based on Vespa. An overview is diagramed below that includes:
- Creating a search engine (Vespa) application in Docker,
- Deploying schema that specify the data structure,
- Processing raw data for ingestion,
- Feeding data into the search engine,
- Querying the search engine and retrieving results. 

<figure>
    <img src="../../resources/pic/vespa_search_tutorial.png" width=700 alt='Tutorial Overview' />
</figure>

Before the tutorial, please:
- Unzip sample_data.zip,
- Turn on your Docker desktop,
- Install the required packages below (<strong>NOTE: You may want to disconnect from the UHG VPN to speed up docker image download</strong>).

In [ ]:
!pip install -r ../../requirements.txt
print('done')

### 1. Search engine creation & deployment

This section demonstrates how to create a Vespa application in Docker and deploy two schema to the application. The search engine configuration files are provided [here](../../resources/application/). Example schema are providered [here](../../resources/application/schemas/). 

In [1]:
from IPython.display import Markdown, display

def printbold(text):
    display(Markdown("**"+text+"**"))

printbold("Import required package and configurations...")
import io, sys, os, json, urllib, datetime, warnings, time
from vespa.deployment import VespaDocker
from vespa.application import Vespa
import pandas as pd

#supress pandas chain assignment warning
pd.options.mode.chained_assignment = None  # default='warn'

**Import required package and configurations...**

/Users/fmordel2/Documents/GitHub/us-res-tutorial-search/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
sys.path.append(os.path.dirname("../"))
from src import constant as settings
from src.ingestion import provider_data_extraction, provider_data_feed
from src.evaluation import query_generation, performance_test

In [3]:
def get_vespa_results(vespa_app, query):
    """
    function to get and display vespa results
    """
    stime = time.time()
    return_response = vespa_app.query(body=query)
    results = return_response.hits
    etime = time.time()

    #show query and results
    printbold("Query (url):")
    print(return_response.url+"?"+urllib.parse.urlencode(query))
    printbold("Response time:")
    print("{:.2f} ms".format(1000*(etime-stime)))
    printbold("Search time:")
    print("{:.2f} ms".format(1000*return_response.json.get('timing').get('searchtime')))
    printbold("Total records:")
    print(return_response.json.get('root').get('coverage').get('documents'))
    printbold("Total matched results:")
    print(return_response.json.get('root').get('fields').get('totalCount'))
    printbold("Top results:")
    print(json.dumps(results, indent=2))

In [4]:
printbold("Deploy the vespa application from local config files...")
app_name = settings.app_name
# config_dir = settings.config_dir
config_dir = "/Users/fmordel2/Documents/GitHub/us-res-tutorial-search/resources/application"

vespa_docker = VespaDocker()
vespa_app = vespa_docker.deploy_from_disk(application_name = app_name,
                                            application_root = config_dir)

printbold("Check if vespa application is up...")
print(vespa_app.get_application_status().ok)
print('done')

**Deploy the vespa application from local config files...**

Waiting for configuration server, 0/300 seconds...
Waiting for configuration server, 5/300 seconds...
Waiting for configuration server, 10/300 seconds...
Waiting for application status, 0/300 seconds...
Waiting for application status, 5/300 seconds...
Waiting for application status, 10/300 seconds...
Waiting for application status, 15/300 seconds...
Waiting for application status, 20/300 seconds...
Waiting for application status, 25/300 seconds...
Finished deployment.


**Check if vespa application is up...**

True
done


After deployment, you can check if the application is up and running at http://localhost:19071/ApplicationStatus (configuration server) and http://localhost:8080/status.html (content server). If you would like to check what is happning in the application, use `docker exec -it --user root tutorialvespa /bin/bash`. You will find the schemas you deployed at `/opt/vespa/var/db/vespa/config_server/serverdb/tenants/default/sessions/2/schemas`.

<strong>Potential causes if you encounter errors during Vespa deployment:</strong>
- You are running another container that uses the same port(s) (500 Server error),
- You have an issue with pulling the Vespa docker image. Please try `docker pull docker.repo1.uhc.com/vespaengine/vespa`.

### 2. Data processing & information extraction

One key process before feeding data into a search engine is data processing and information extraction. This section demonstrates a simple pipeline to 
- clean raw data (e.g., removing expired records), 
- extract information (e.g., extracting required data fields), and 
- save data in the format specified by the schema. 

The pipeline is described [here](./ingestion/provider_data_extraction.py).

In [5]:
printbold("Perform data processing & information extraction...")
provider_data_extraction.main_process(settings)
print("done")

**Perform data processing & information extraction...**

Process file: ../../data/organization_sample_data.json

Void record during process: providerData (ignore this document 7055809-420680448-T-855151-L).
Have processed 1000 records.
Void record during process: providerData (ignore this document 7690140-832558614-T-3045912-D).
Have processed 2000 records.
Void record during process: providerData (ignore this document 7617794-320452611-T-3267557-L).
Have processed 3000 records.
Void record during process: providerData (ignore this document 7422788-541077240-T-3064459-L).
Have processed 4000 records.
Processed a total of 4046 records.
Data frame shape: (4042, 22)

Save data frame.

-----------------
Process file: ../../data/practitioner_sample_data.json

Have processed 1000 records.
Have processed 2000 records.
Have processed 3000 records.
Have processed 4000 records.
Void record during process: providerData (ignore this document 7431749-462799683-T-2924831-H).
Have processed 5000 records.
Void record during process: providerData (ignore thi

#### 2.1 Exploratory data analysis & inspection
In most cases real-world data are noisy and require (manual) inspection to avoid garbage-in-garbage-out problems. In general, exploratory data analysis (not done in this tutorial) will be performed before schema design and data processing to identify data issues and inform solutions. Another data inspection will be performed before data ingestion to clear issues such as null values, invalid records, and incompatible formats.  

This section demonstrates one data inspection example. 

In [8]:
printbold("Load processed data and performed data inspection...")
data_file = settings.local_indir + settings.all_outfiles[0] #0. organization data, 1. practitioner data
df = pd.read_pickle(data_file)

#data overview
print(df.head())
print(df.columns)

#data inspection example
temp_df = df[['doc_expire_date']] #doc_expire_date stores record expire date
print(temp_df.head(10).to_string(index=False))
print("---------------------------------")

temp_df['doc_expire_date_dttm'] = [datetime.datetime.fromtimestamp(x) for x in temp_df['doc_expire_date']]
print(temp_df.head(10).to_string(index=False))
print("---------------------------------")

num_expired_records = sum(temp_df['doc_expire_date_dttm'] < datetime.datetime.strptime('2023-01-01','%Y-%m-%d'))
print("Number of expired records: {} ({:.2f}%)".format(num_expired_records, 100*num_expired_records/(len(temp_df))))
print("---------------------------------")
print('done')

**Load processed data and performed data inspection...**

  enterprise_provider_id                  generated_key  doc_expire_date  \
0                2376287  2376287-351520388-T-1635227-L     253402214400   
1                3260346   3260346-273899821-T-268376-L     253402214400   
2                3691691  3691691-830344890-T-2372076-L     253402214400   
3                 822935      822935-61064225-T-94898-D     253402214400   
4                 237359   237359-232450112-T-3228242-H       1631487600   

  first_name middle_name last_name                          org_name  \
0                                              chiropractic wellness   
1                                                 st josephs medical   
2                                     michigan pathology specialists   
3                                               faire harbour ob/gyn   
4                                   planned parenthood of lehigh vly   

  prov_type_code organization_type_code  address_id  ... county_name  \
0              o                    03

### 3. Data ingestion
This section demonstrates how to feed data into the search engine application. The vespa application supports point-by-point ingestion or batch ingestion from a dictionary list or a pandas data frame. You can also feed data using curl put actions.

The data ingestion pipeline is described [here](./ingestion/provider_data_feed.py).

In [9]:
printbold("Perform data ingestion...")

#feed organization data
settings.schema_name = 'organization'
provider_data_feed.main_process(settings)

#feed practitioner data
settings.schema_name = 'practitioner'
provider_data_feed.main_process(settings)
print("done")

**Perform data ingestion...**

Feed data from file: ../../data/organization_sample_data.pkl
Successful documents fed: 1000/1000.
Batch progress: 1/5.
Successful documents fed: 1000/1000.
Batch progress: 2/5.
Successful documents fed: 1000/1000.
Batch progress: 3/5.
Successful documents fed: 1000/1000.
Batch progress: 4/5.
Successful documents fed: 42/42.
Batch progress: 5/5.
File process time: 20.400991201400757 secs.
Execution time: 20.402027130126953 secs.
Feed data from file: ../../data/practitioner_sample_data.pkl
Successful documents fed: 1000/1000.
Batch progress: 1/20.
Successful documents fed: 1000/1000.
Batch progress: 2/20.
Successful documents fed: 1000/1000.
Batch progress: 3/20.
Successful documents fed: 1000/1000.
Batch progress: 4/20.
Successful documents fed: 1000/1000.
Batch progress: 5/20.
Successful documents fed: 1000/1000.
Batch progress: 6/20.
Successful documents fed: 1000/1000.
Batch progress: 7/20.
Successful documents fed: 1000/1000.
Batch progress: 8/20.
Successful documents fed: 1000/1000

In [10]:
vespa_app = Vespa(url = settings.vespa_url_local) #connect to local host
schema_name = "organization" #["organization", "practitioner"]
return_fields = "*"

printbold("Data ingestion inspection for schema {}...".format(schema_name))
query = {
    'yql': 'select {} from sources {} where true;'.format(return_fields, schema_name),
    'type': 'any',
    'presentation.timing': True,
    'hits': "1"
}

#show query and results
get_vespa_results(vespa_app, query)

**Data ingestion inspection for schema organization...**

**Query (url):**

http://localhost:8080/search/?yql=select+%2A+from+sources+organization+where+true%3B&type=any&presentation.timing=True&hits=1


**Response time:**

266.93 ms


...

4042


**Total matched results:**

4042


**Top results:**

[
  {
    "id": "id:organization:organization::3691691-830344890-T-2372076-L",
    "relevance": 0.0,
    "source": "provider_content",
    "fields": {
      "sddocname": "organization",
      "documentid": "id:organization:organization::3691691-830344890-T-2372076-L",
      "csp_contract": {
        "911": 99991231
      },
      "unet_contract": {
        "21613-lh-o": 99991231,
        "21613-li-o": 99991231,
        "21613-lg-o": 99991231,
        "21613-p3-o": 99991231,
        "21613-c0-o": 99991231,
        "21613-c2-o": 99991231,
        "21613-c1-o": 99991231,
        "21613-ea-o": 99991231,
        "21613-eb-o": 99991231,
        "21613-s7-o": 99991231,
        "21613-s8-o": 99991231,
        "21613-s9-o": 99991231,
        "21613-ee-o": 99991231,
        "21613-ef-o": 99991231,
        "21613-ec-o": 99991231,
        "21613-ed-o": 99991231,
        "21613-ei-o": 99991231,
        "210-e0-": 99991231,
        "21613-eg-o": 99991231,
        "21613-s0-o": 99991231,
        "216

### 4. Query examples & performance test
The search engine is ready to use after data ingestion. This section demonstrates query examples for testing the deployed application.

In [11]:
printbold("Set global configurations...")
settings.num_hit = 10 #number of returned results
print('done')

**Set global configurations...**

done


In [12]:
vespa_app = Vespa(url = settings.vespa_url_local) #connect to local host
schema_name = "practitioner" #["organization", "practitioner"]
ranking_profile = "org_bm25" if schema_name.startswith("o") else "prov_bm25"
return_fields = "*"
query_term = "john smith"

printbold("Query example 1: text matching for schema {}...".format(schema_name))
query = {
    'yql': 'select {} from sources {} where userQuery();'.format(return_fields, schema_name),
    'query': query_term,
    'ranking': ranking_profile,
    'type': 'any',
    'presentation.timing': True,
    'hits': settings.num_hit 
}

#show query and results
get_vespa_results(vespa_app, query)

**Query example 1: text matching for schema practitioner...**

**Query (url):**

http://localhost:8080/search/?yql=select+%2A+from+sources+practitioner+where+userQuery%28%29%3B&query=john+smith&ranking=prov_bm25&type=any&presentation.timing=True&hits=10


**Response time:**

144.76 ms


...

19558


**Total matched results:**

513


**Top results:**

[
  {
    "id": "id:practitioner:practitioner::1821838-752931720-T-1130898-L",
    "relevance": 15.71983775959675,
    "source": "provider_content",
    "fields": {
      "sddocname": "practitioner",
      "documentid": "id:practitioner:practitioner::1821838-752931720-T-1130898-L",
      "cosmos_contract": {
        "pnx-709": 99991231,
        "etx-709": 99991231
      },
      "csp_contract": {
        "301": 99991231,
        "302": 99991231,
        "303": 99991231,
        "304": 99991231,
        "306": 99991231,
        "307": 99991231
      },
      "unet_contract": {
        "42580-r0-": 99991231,
        "42580-s2-o": 99991231,
        "42580-s1-o": 99991231,
        "42580-s0-o": 99991231,
        "42580-ec-o": 99991231,
        "42580-sg-o": 99991231,
        "42580-sh-o": 99991231,
        "42580-ed-o": 99991231,
        "42580-ea-o": 99991231,
        "42580-si-o": 99991231,
        "42580-eb-o": 99991231,
        "420-d0-": 99991231,
        "42580-c0-": 99991231,
      

In [13]:
vespa_app = Vespa(url = settings.vespa_url_local) #connect to local host
schema_name = "organization" #["organization", "practitioner"]
ranking_profile = "org_bm25" if schema_name.startswith("o") else "prov_bm25"
return_fields = "*"
query_field = "organization"
query_term = '"wellness clinics"'

printbold("Query example 2: text matching for schema {} on a specific fieldset: {}...".format(schema_name, query_field))
query = {
    'yql': 'select {} from sources {} where {} contains {};'.format(return_fields, schema_name, 
                                                                    query_field, query_term),
    'ranking': ranking_profile,
    'type': 'any',
    'presentation.timing': True,
    'hits': settings.num_hit
}

#show query and results
get_vespa_results(vespa_app, query)

**Query example 2: text matching for schema organization on a specific fieldset: organization...**

**Query (url):**

http://localhost:8080/search/?yql=select+%2A+from+sources+organization+where+organization+contains+%22wellness+clinics%22%3B&ranking=org_bm25&type=any&presentation.timing=True&hits=10


**Response time:**

35.14 ms


...

4042


**Total matched results:**

2


**Top results:**

[
  {
    "id": "id:organization:organization::5809943-474339905-T-1554078-D",
    "relevance": 9.180158590857811,
    "source": "provider_content",
    "fields": {
      "sddocname": "organization",
      "documentid": "id:organization:organization::5809943-474339905-T-1554078-D",
      "specialty_org": {
        "384-acn-p": 99991231,
        "384-uhn-p": 99991231
      },
      "contract_org": {
        "uhn-p-p": 99991231
      },
      "national_taxonomy": {
        "261qh0100x": 99991231
      },
      "enterprise_provider_id": "5809943",
      "generated_key": "5809943-474339905-T-1554078-D",
      "doc_expire_date": 253402214400,
      "org_name": "vancouver wellness clinic",
      "prov_type_code": "o",
      "organization_type_code": "050",
      "address_id": 1554078,
      "address_line": "304 e 37th st",
      "city_name": "vancouver",
      "county_name": "clark",
      "state_code": "wa",
      "zipcode": "98663",
      "geocode": {
        "lat": 45.648575,
        "lng

In [14]:
vespa_app = Vespa(url = settings.vespa_url_local) #connect to local host
schema_name = "organization" #["organization", "practitioner"]
ranking_profile = "org_geo_filter" if schema_name.startswith("o") else "prov_geo_filter"
return_fields = "*"
query_term = 'little clinic'

#geo search
latitude = 39.592040
longitude = -104.919420
radius = 25

printbold("Query example 3: text matching with geo search for schema {}...".format(schema_name))
query = {
    'yql': 'select {} from sources {} where userQuery() and geoLocation(geocode, {}, {}, "{} miles");'.format(
        return_fields, schema_name, latitude, longitude, radius),
    'query': query_term,
    'ranking': ranking_profile,
    'list'
    'type': 'any',
    'presentation.timing': True,
    'hits': settings.num_hit
}

#show query and results
get_vespa_results(vespa_app, query)

**Query example 3: text matching with geo search for schema organization...**

**Query (url):**

http://localhost:8080/search/?yql=select+%2A+from+sources+organization+where+userQuery%28%29+and+geoLocation%28geocode%2C+39.59204%2C+-104.91942%2C+%2225+miles%22%29%3B&query=little+clinic&ranking=org_geo_filter&listtype=any&presentation.timing=True&hits=10


**Response time:**

52.69 ms


...

4042


**Total matched results:**

3


**Top results:**

[
  {
    "id": "id:organization:organization::2762037-841494091-T-2023399-D",
    "relevance": 7.315224451818665,
    "source": "provider_content",
    "fields": {
      "sddocname": "organization",
      "documentid": "id:organization:organization::2762037-841494091-T-2023399-D",
      "cosmos_contract": {
        "mnr-135": 99991231,
        "pnx-662": 99991231,
        "mnr-112": 99991231,
        "mnr-134": 99991231,
        "mnr-133": 99991231,
        "mnr-132": 99991231,
        "evc-7": 99991231,
        "evc-47": 99991231,
        "evc-684": 99991231,
        "mnr-136": 99991231
      },
      "unet_contract": {
        "5440-ei-o": 99991231,
        "5440-eg-o": 99991231,
        "5440-eh-o": 99991231,
        "5440-ee-o": 99991231,
        "5440-ef-o": 99991231,
        "5440-ec-o": 99991231,
        "5440-ed-o": 99991231,
        "71002-r0-": 99991231,
        "5440-ea-o": 99991231,
        "5440-eb-o": 99991231,
        "5440-p3-o": 99991231,
        "5440-r0-": 99991231,

In [15]:
vespa_app = Vespa(url = settings.vespa_url_local) #connect to local host
schema_name = "organization" #["organization", "practitioner"]
ranking_profile = "geo_ranking"
return_fields = "*"
query_term = 'little clinic'

#geo search
latitude = 39.592040
longitude = -104.919420
radius = 25

printbold("Query example 4: text matching with geo ranking for schema {}...".format(schema_name))
query = {
    'yql': 'select {} from sources {} where userQuery() and geoLocation(geocode, {}, {}, "{} miles");'.format(
        return_fields, schema_name, latitude, longitude, radius),
    'query': query_term,
    'ranking': ranking_profile,
    'type': 'any',
    'presentation.timing': True,
    'hits': settings.num_hit
}

#show query and results
get_vespa_results(vespa_app, query)

**Query example 4: text matching with geo ranking for schema organization...**

**Query (url):**

http://localhost:8080/search/?yql=select+%2A+from+sources+organization+where+userQuery%28%29+and+geoLocation%28geocode%2C+39.59204%2C+-104.91942%2C+%2225+miles%22%29%3B&query=little+clinic&ranking=geo_ranking&type=any&presentation.timing=True&hits=10


**Response time:**

28.28 ms


...

4042


**Total matched results:**

3


**Top results:**

[
  {
    "id": "id:organization:organization::3137775-271804731-T-2599261-H",
    "relevance": -7.397113747884665,
    "source": "provider_content",
    "fields": {
      "sddocname": "organization",
      "documentid": "id:organization:organization::3137775-271804731-T-2599261-H",
      "csp_contract": {
        "918": 99991231
      },
      "unet_contract": {
        "5440-ei-o": 99991231,
        "5440-eg-o": 99991231,
        "5440-eh-o": 99991231,
        "5440-ee-o": 99991231,
        "5440-ef-o": 99991231,
        "5440-ec-o": 99991231,
        "71002-r0-": 99991231,
        "5440-ed-o": 99991231,
        "5440-lg-o": 99991231,
        "5440-ea-o": 99991231,
        "5440-eb-o": 99991231,
        "5440-li-o": 99991231,
        "5440-lh-o": 99991231,
        "5440-p3-o": 99991231,
        "5440-c1-": 99991231,
        "50-e0-": 99991231,
        "5440-sg-o": 99991231,
        "5440-sh-o": 99991231,
        "5440-si-o": 99991231,
        "5440-s2-o": 99991231,
        "5440-s1-o

In [16]:
vespa_app = Vespa(url = settings.vespa_url_local) #connect to local host
schema_name = "organization" #["organization", "practitioner"]
ranking_profile = "org_geo_filter" if schema_name.startswith("o") else "prov_geo_filter"
return_fields = "*"
query_term = 'clinic' #also try 'clinics'

#geo search
latitude = 39.592040
longitude = -104.919420
radius = 25

#specialty filter
specialty_org = '"015-uhn-p"'
spl_expire_date = 20230401

printbold("Query example 5: text matching + geo search + structured filters for schema...".format(schema_name))
query = {
    'yql': 'select {} from sources {} where (userQuery() and specialty_org contains sameElement(key contains {},  value > {}) and geoLocation(geocode, {}, {}, "{} miles"));'.format(
        return_fields, schema_name, specialty_org, spl_expire_date, latitude, longitude, radius),
    'query': query_term,
    'ranking': ranking_profile,
    'type': 'any',
    'presentation.timing': True,
    'hits': settings.num_hit
}

#show query and results
get_vespa_results(vespa_app, query)

**Query example 5: text matching + geo search + structured filters for schema...**

**Query (url):**

http://localhost:8080/search/?yql=select+%2A+from+sources+organization+where+%28userQuery%28%29+and+specialty_org+contains+sameElement%28key+contains+%22015-uhn-p%22%2C++value+%3E+20230401%29+and+geoLocation%28geocode%2C+39.59204%2C+-104.91942%2C+%2225+miles%22%29%29%3B&query=clinic&ranking=org_geo_filter&type=any&presentation.timing=True&hits=10


**Response time:**

62.58 ms


...

4042


**Total matched results:**

1


**Top results:**

[
  {
    "id": "id:organization:organization::2762037-841494091-T-2023399-D",
    "relevance": 7.315224451818665,
    "source": "provider_content",
    "fields": {
      "sddocname": "organization",
      "documentid": "id:organization:organization::2762037-841494091-T-2023399-D",
      "cosmos_contract": {
        "mnr-135": 99991231,
        "pnx-662": 99991231,
        "mnr-112": 99991231,
        "mnr-134": 99991231,
        "mnr-133": 99991231,
        "mnr-132": 99991231,
        "evc-7": 99991231,
        "evc-47": 99991231,
        "evc-684": 99991231,
        "mnr-136": 99991231
      },
      "unet_contract": {
        "5440-ei-o": 99991231,
        "5440-eg-o": 99991231,
        "5440-eh-o": 99991231,
        "5440-ee-o": 99991231,
        "5440-ef-o": 99991231,
        "5440-ec-o": 99991231,
        "5440-ed-o": 99991231,
        "71002-r0-": 99991231,
        "5440-ea-o": 99991231,
        "5440-eb-o": 99991231,
        "5440-p3-o": 99991231,
        "5440-r0-": 99991231,

#### 4.1 Performance test

Several evaluations will be performed to assess a search engine's capacity before production. In my opinion, there are at least two major evaluations:
- <strong>Performance test</strong> that assesses a search engine's query latency (speed), and 
- <strong>Search evaluation</strong> that measures the quality of returned results (i.e., how relevant they are to a user query).

Search evaluation usually relies on a set of gold-standard (or reference-standard) query-clicked url pairs derived from user search behavior data, while performance test only requires a set of queries either provided by clients or generated randomly from data. 

This section demonstrates a pipeline for performance test, including:
- Generating [pseudo queries](../../query/) from data, and
- Conducting performance test and creating a performance report. 

The pipeline is described [here](./evaluation/).

In [17]:
vespa_app = Vespa(url = settings.vespa_url_local) #connect to local host
settings.schema_name = "organization"#["organization", "practitioner"]

#generate pseudo queries
printbold("Generate pseudo queries for schema {}...".format(settings.schema_name))
query_generation.main_process(settings)
print('done')

#performance test
printbold("Performance test for schema {}...".format(settings.schema_name))
performance_test.main_process(settings)
print('done')

**Generate pseudo queries for schema organization...**

Select data points from file: ../../data/organization_sample_data.pkl
done


**Performance test for schema organization...**

Perform Vespa benchmark test (30 secs)...
Starting clients...
Stopping clients
Clients stopped.
.....
Clients Joined.
*** HTTP keep-alive statistics ***
connection reuse count -- 37820
***************** Benchmark Summary *****************
clients:                       5
ran for:                      30 seconds
cycle time:                    0 ms
lower response limit:          0 bytes
skipped requests:              0
failed requests:               0
successful requests:       37725
cycles not held:           37725
minimum response time:      1.23 ms
maximum response time:     21.13 ms
average response time:      3.92 ms
25   percentile:              3.00 ms
50   percentile:              3.60 ms
75   percentile:              4.50 ms
90   percentile:              5.60 ms
95   percentile:              6.50 ms
98   percentile:              8.00 ms
99   percentile:              9.30 ms
99.5 percentile:             10.90 ms
99.6 percentile:             11.40 ms
99.7 percentile:             1

### 5. Scaling up - cloud-based web service

One solution to scale up is to migrate the service to the cloud (e.g., to an AWS Kubernetes cluster). Although you need additional steps for cluster configuration, the major steps (EDA, schema deployment, data processing, ingestion) are identical.  

You can try the cloud-based search engine we developed for UHC provider search [here](https://dev.tools.search.optum.com/#/unified-provider). 

### 6. Application management

If the vespa application is no longer needed, you can run the following code to delete the container. Alternatively you can delete the container in the Docker desktop. 

In [18]:
printbold("Stop the vespa application and delete the container...")
if "vespa_docker" in locals() and vespa_docker:
    vespa_docker.container.stop(timeout=15)
    vespa_docker.container.remove()
else:
    print("Container variable not found. Please delete the container in Docker desktop.")
print("done")

**Stop the vespa application and delete the container...**

done
